In [1]:
#coding:utf-8
import numpy as np
dataset_filename="affinity_dataset.txt"
#加载数据
dataset=np.loadtxt(dataset_filename)
dataset[:5]

array([[ 0.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  1.,  1.,  1.],
       [ 0.,  1.,  0.,  0.,  1.]])

In [3]:
#一行表示一个用户对五个商品的购买情况  1.表示购买  2.表示未买
n_samples,n_features=dataset.shape
print('{0}条记录，每条记录{1}个特征项'.format(n_samples,n_features))

100条记录，每条记录5个特征项


(100, 5)

In [4]:
#定义特征集合
features=['bread','milk','cheese','apple','banana']
features

['bread', 'milk', 'cheese', 'apple', 'banana']

# 目标一
##### 用户购买了apple  可能也想买banana

In [5]:
#首先有多少行有这条规则的前提  用户购买过apple
num_apple_purchase=0
for sample in dataset:
    if sample[3]==1:
        num_apple_purchase+=1
print(num_apple_purchase)

36


In [6]:
#下面可以计算这条规则(用户购买了apple  那它也可能购买banana)的应验数与无无效
rule_valid=0.0   #规则的应验数
rule_invalid=0.0   #规则的无效数
for sample in dataset:
    if sample[3]==1:
        if sample[4]==1:
            rule_valid+=1
        else:
            rule_invalid+=1
print("{0}无效的次数".format(rule_invalid))
print("{0}应验的次数".format(rule_valid))

15.0无效的次数
21.0应验的次数


In [7]:
#支持度
support=rule_valid
support

21.0

In [11]:
#  用户购买了apple  那他可能购买了banana  这条规则 置信度  支持度/前提条件出现的次数
confidence=rule_valid/num_apple_purchase  
print('支持度{0}置信度{1:.3f}'.format(support,confidence))   #小数点后保留三位

支持度21.0置信度0.583


# 有了上面的案例，用于理解支持度和置信度
计算支持度和置信度要求得到：  有效规则数  无效规则数   前提条件出现的次数
下面开始所有的规则的支持度和置信度的计算

In [12]:
from collections import defaultdict
#数据格式采用字典   {键（前提）:值（数值）}
valid_rules=defaultdict(int)   #每条规则的支持度
invalid_rules=defaultdict(int)  #每条规则的无效数
#数据格式采用字典
num_rule_occurances=defaultdict(float)   #规则前提条件出现的次数

In [13]:
for sample in dataset:
    #premise前提条件
    for premise in range(n_features):
        if sample[premise]==0: continue  #没有购买条件不成立  直接继续循环
        num_rule_occurances[premise]+=1.0   #出现了一次前提条件
        for conclusion in range(n_features):  #计算他的支持度和无效数
            if premise==conclusion:
                continue
            if sample[conclusion]==1:
                valid_rules[(premise,conclusion)]+=1
            else:
                invalid_rules[(premise,conclusion)]+=1

In [14]:
print('{0}无效规则'.format(valid_rules))
print('{0}有效规则'.format(invalid_rules))

defaultdict(<class 'int'>, {(2, 3): 25, (2, 4): 27, (3, 2): 25, (3, 4): 21, (4, 2): 27, (4, 3): 21, (0, 1): 14, (0, 3): 5, (1, 0): 14, (1, 3): 9, (3, 0): 5, (3, 1): 9, (0, 2): 4, (2, 0): 4, (1, 4): 19, (4, 1): 19, (0, 4): 17, (4, 0): 17, (1, 2): 7, (2, 1): 7})无效规则
defaultdict(<class 'int'>, {(2, 0): 37, (2, 1): 34, (3, 0): 31, (3, 1): 27, (4, 0): 42, (4, 1): 40, (0, 2): 23, (0, 4): 10, (1, 2): 39, (1, 4): 27, (3, 2): 11, (3, 4): 15, (0, 1): 13, (2, 4): 14, (1, 0): 32, (1, 3): 37, (4, 2): 32, (4, 3): 38, (0, 3): 22, (2, 3): 16})有效规则


In [15]:
#下一步  计算支持度与置信度
support=valid_rules
confidence=defaultdict(float)
for premise,conclusion in valid_rules.keys():
    confidence[(premise,conclusion)]=valid_rules[(premise,conclusion)]/num_rule_occurances[premise]
print('所有规则的支持度',support)
print('所有规则的置信度',confidence)

所有规则的支持度 defaultdict(<class 'int'>, {(2, 3): 25, (2, 4): 27, (3, 2): 25, (3, 4): 21, (4, 2): 27, (4, 3): 21, (0, 1): 14, (0, 3): 5, (1, 0): 14, (1, 3): 9, (3, 0): 5, (3, 1): 9, (0, 2): 4, (2, 0): 4, (1, 4): 19, (4, 1): 19, (0, 4): 17, (4, 0): 17, (1, 2): 7, (2, 1): 7})
所有规则的置信度 defaultdict(<class 'float'>, {(2, 3): 0.6097560975609756, (2, 4): 0.6585365853658537, (3, 2): 0.6944444444444444, (3, 4): 0.5833333333333334, (4, 2): 0.4576271186440678, (4, 3): 0.3559322033898305, (0, 1): 0.5185185185185185, (0, 3): 0.18518518518518517, (1, 0): 0.30434782608695654, (1, 3): 0.1956521739130435, (3, 0): 0.1388888888888889, (3, 1): 0.25, (0, 2): 0.14814814814814814, (2, 0): 0.0975609756097561, (1, 4): 0.41304347826086957, (4, 1): 0.3220338983050847, (0, 4): 0.6296296296296297, (4, 0): 0.288135593220339, (1, 2): 0.15217391304347827, (2, 1): 0.17073170731707318})


In [16]:
#将上面的结果格式化一下
for premise,conclusion in confidence:
    #求出规则前提条件的商品名
    premise_name=features[premise]
    #求出规则条件的结论对应的商品名
    conclusion_name=features[conclusion]
    print('规则：如果一个人购买了{0}他也会购买{1}'.format(premise_name,conclusion_name))
    print('这条规则的支持度为:{0}'.format(support[(premise,conclusion)] ) )
    print('这条规则的置信度为:{0}'.format(confidence[(premise,conclusion)] ) )
    

规则：如果一个人购买了cheese他也会购买apple
这条规则的支持度为:25
这条规则的置信度为:0.6097560975609756
规则：如果一个人购买了cheese他也会购买banana
这条规则的支持度为:27
这条规则的置信度为:0.6585365853658537
规则：如果一个人购买了apple他也会购买cheese
这条规则的支持度为:25
这条规则的置信度为:0.6944444444444444
规则：如果一个人购买了apple他也会购买banana
这条规则的支持度为:21
这条规则的置信度为:0.5833333333333334
规则：如果一个人购买了banana他也会购买cheese
这条规则的支持度为:27
这条规则的置信度为:0.4576271186440678
规则：如果一个人购买了banana他也会购买apple
这条规则的支持度为:21
这条规则的置信度为:0.3559322033898305
规则：如果一个人购买了bread他也会购买milk
这条规则的支持度为:14
这条规则的置信度为:0.5185185185185185
规则：如果一个人购买了bread他也会购买apple
这条规则的支持度为:5
这条规则的置信度为:0.18518518518518517
规则：如果一个人购买了milk他也会购买bread
这条规则的支持度为:14
这条规则的置信度为:0.30434782608695654
规则：如果一个人购买了milk他也会购买apple
这条规则的支持度为:9
这条规则的置信度为:0.1956521739130435
规则：如果一个人购买了apple他也会购买bread
这条规则的支持度为:5
这条规则的置信度为:0.1388888888888889
规则：如果一个人购买了apple他也会购买milk
这条规则的支持度为:9
这条规则的置信度为:0.25
规则：如果一个人购买了bread他也会购买cheese
这条规则的支持度为:4
这条规则的置信度为:0.14814814814814814
规则：如果一个人购买了cheese他也会购买bread
这条规则的支持度为:4
这条规则的置信度为:0.0975609756097561
规则：如果一个人购买了milk他也会购买banana
这条规则的支持度为:19
这条规则的

In [17]:
'''
定义一个函数输出指定规则的支持度，置信度
premise  前提的索引
conclusion  结论的索引
support  支持度的字典
confidence 置信度的字典
feature  特征数组
'''
def print_rule( premise,conclusion,support,confidence,features):
    #求出规则前提条件商品名
    premise_name=features[premise]
    #求出规则条件的结论对应的商品名
    conclusion_name=features[conclusion]
    print('如果他买了{0}所以他还会买{1}'.format(premise_name,conclusion_name))
    print('这条规则的支持度',support[(premise,conclusion) ] )
    print('这条规则的置信度',confidence[(premise,conclusion) ] )
    
    
#测试
premise=1
conclusion=3
print_rule(premise,conclusion,support,confidence,features)

如果他买了milk所以他还会买apple
这条规则的支持度 9
这条规则的置信度 0.1956521739130435


In [18]:
#  对支持度进行排序

#先排序输出
print(list(support.items() ) )
#排好序后存到字典
from operator import itemgetter
#  根据这个置信度排序
sorted_support=sorted(support.items(),key=itemgetter(1),reverse=True)
#输出前五名
for index in range(5):
    print('Rule #{0}'.format(index+1))
    (premise,conclusion)=sorted_support[index][0]  #取出键
    print_rule(premise,conclusion,support,confidence,features)

[((2, 3), 25), ((2, 4), 27), ((3, 2), 25), ((3, 4), 21), ((4, 2), 27), ((4, 3), 21), ((0, 1), 14), ((0, 3), 5), ((1, 0), 14), ((1, 3), 9), ((3, 0), 5), ((3, 1), 9), ((0, 2), 4), ((2, 0), 4), ((1, 4), 19), ((4, 1), 19), ((0, 4), 17), ((4, 0), 17), ((1, 2), 7), ((2, 1), 7)]
Rule #1
如果他买了cheese所以他还会买banana
这条规则的支持度 27
这条规则的置信度 0.6585365853658537
Rule #2
如果他买了banana所以他还会买cheese
这条规则的支持度 27
这条规则的置信度 0.4576271186440678
Rule #3
如果他买了cheese所以他还会买apple
这条规则的支持度 25
这条规则的置信度 0.6097560975609756
Rule #4
如果他买了apple所以他还会买cheese
这条规则的支持度 25
这条规则的置信度 0.6944444444444444
Rule #5
如果他买了apple所以他还会买banana
这条规则的支持度 21
这条规则的置信度 0.5833333333333334


In [19]:
#置信度

#先排序输出
print(list(confidence.items() ) )
#排好序后存到字典
from operator import itemgetter
#  根据这个置信度排序
sorted_confidence=sorted(confidence.items(),key=itemgetter(1),reverse=True)
#输出前五名
for index in range(5):
    print('Rule #{0}'.format(index+1))
    (premise,conclusion)=sorted_confidence[index][0]  #取出键
    print_rule(premise,conclusion,support,confidence,features)

[((2, 3), 0.6097560975609756), ((2, 4), 0.6585365853658537), ((3, 2), 0.6944444444444444), ((3, 4), 0.5833333333333334), ((4, 2), 0.4576271186440678), ((4, 3), 0.3559322033898305), ((0, 1), 0.5185185185185185), ((0, 3), 0.18518518518518517), ((1, 0), 0.30434782608695654), ((1, 3), 0.1956521739130435), ((3, 0), 0.1388888888888889), ((3, 1), 0.25), ((0, 2), 0.14814814814814814), ((2, 0), 0.0975609756097561), ((1, 4), 0.41304347826086957), ((4, 1), 0.3220338983050847), ((0, 4), 0.6296296296296297), ((4, 0), 0.288135593220339), ((1, 2), 0.15217391304347827), ((2, 1), 0.17073170731707318)]
Rule #1
如果他买了apple所以他还会买cheese
这条规则的支持度 25
这条规则的置信度 0.6944444444444444
Rule #2
如果他买了cheese所以他还会买banana
这条规则的支持度 27
这条规则的置信度 0.6585365853658537
Rule #3
如果他买了bread所以他还会买banana
这条规则的支持度 17
这条规则的置信度 0.6296296296296297
Rule #4
如果他买了cheese所以他还会买apple
这条规则的支持度 25
这条规则的置信度 0.6097560975609756
Rule #5
如果他买了apple所以他还会买banana
这条规则的支持度 21
这条规则的置信度 0.5833333333333334
